In [1]:
import pandas as pd
import numpy as np 
import time
from collections import Counter
import re
import math
from gensim.models import KeyedVectors
from pprint import pprint


MAX_LEAFS = 256



class Tree(object):
    
    def __init__(self, tree_hierarchy, clusters_predict=[], mode="sklearn", sentences_all_classes=None, true_classes_all=None):
        """
        @param mode : Two possible values
            - "FBE" : Tree object for Feedback Explorer output
            - "sklearn" : Tree object for scikit learn output
        
        @param sentences_all_classes : List of all possible classes occuring in the sentences file (only for mode FBE)
        @param true_labels_all : All occuring true labels (mesh codes) of the documents/abstracts
        """
        self.tree = None
        if mode in ["sklearn", "FBE"]:
            self.mode = mode
        else:
            raise ValueError("Provided mode '{}' is not supported".format(mode))
        self.tree_hierarchy = tree_hierarchy # pandas dataframe with tree structure coming from hierarchical clustering
        self.n_nodes = 0 # updated by calling self.count_nodes()
        self.n_leafs = 0 # updated by calling self.count_leafs()
        self.temp_n_leafs = 1 # In mode 'FBE' helps to construct the tree with the right number of nodes
        self.clusters_predict = clusters_predict # predicted cluster for each document
        self.unique_cluster_predict = list(set(clusters_predict)) # list of all classes to calculate performance metrices
        self.leaf_nodes = [] # list of all leaf nodes
        self.sentences_all_classes = sentences_all_classes # List of all classes occuring in sentences (phrases.parquet)
        self.true_classes_documents = true_classes_all.values.tolist() # list of true labels (mesh codes) in the abstracts
        self.true_classes_documents_unique = list(set(true_classes_all)) # all possible occuring true labels (mesh codes) in the abstracts
        self.precision_all_nodes = [] # macro
        self.precision_all_nodes_weighted = []
        self.precision_all_nodes_weights = 0
        self.precision_macro = None 
        self.precision_micro = None
        self.recall_all_classes = []
        self.recall_all_classes_weighted = []
        self.recall_macro = None
        self.recall_micro = None
        self.F1_macro = None
        self.F1_micro = None
        self.maxDepth = 0
        self.temp_max_occ_class_in_cluster = 0
        self.temp_max_doc_perClass_inCluster = 0
        
        
    def _build_tree(self, node, current_depth=None):
        if self.mode == "sklearn":
            if node.node_id in self.tree_hierarchy["node_id"].values: # if node not leaf
                treeChildren = self.tree_hierarchy[self.tree_hierarchy["node_id"] == node.node_id]
                node.add_child(Node(Id=treeChildren["left"].values[0], depth=node.depth + 1, parent=node))
                node.add_child(Node(Id=treeChildren["right"].values[0], depth=node.depth + 1, parent=node))
                self._build_tree(node.children[0])
                self._build_tree(node.children[1])
            else:
                return node
            return node
            #print(node)
#            #print("\tcurrent_depth: {}; temp_nleafs: {}".format(current_depth, self.temp_n_leafs))
#            if node.depth == current_depth and self.temp_n_leafs < MAX_LEAFS:
#                if node.node_id in self.tree_hierarchy["node_id"].values: # if node not leaf
#                    treeChildren = self.tree_hierarchy[self.tree_hierarchy["node_id"] == node.node_id]
#                    self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
#                    node.add_child(Node(Id=treeChildren["left"].values[0], depth=node.depth + 1, parent=node))
#                    self.temp_n_leafs += 1 # creates new child
#                    node.add_child(Node(Id=treeChildren["right"].values[0], depth=node.depth + 1, parent=node))
#                    self.temp_n_leafs += 1 # creates new child
#            else:
#                if len(node.children) == 2 and self.temp_n_leafs < MAX_LEAFS:
#                    self._build_tree(node.children[0], current_depth)
#                    self._build_tree(node.children[1], current_depth)
#            return node
        elif self.mode == "FBE": 
            # Only create node if node is in current depth level
            if node.depth == current_depth and self.temp_n_leafs < MAX_LEAFS:
                treeChildren = self.tree_hierarchy.iloc[node.node_id].children
                #print("\t{}".format(node))
                #print("tree children:")
                #print(treeChildren)
                #print()
                # FBE tree is not a perfect binary tree, some nodes don't create children any more
                if len(treeChildren) > 0:
                    cluster_child_one = self.tree_hierarchy.iloc[treeChildren[0]].filterValue[0]
                    cluster_child_two = self.tree_hierarchy.iloc[treeChildren[1]].filterValue[0]
                #    print("c1: {}, c2: {}".format(cluster_child_one, cluster_child_two))
                    # Some nodes from nodes.json are empty: no sentences is going through them
                    # Only create node in tree when there is a sentence running through it
                    if cluster_child_one in self.sentences_all_classes:
                #        print("\c1 in class")
                        self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
                        node.add_child(Node(Id=treeChildren[0], depth=node.depth + 1, parent=node, cluster_label=cluster_child_one))
                        self.temp_n_leafs += 1
                        if cluster_child_two in sentences_all_classes:
                #            print("\tc1 and c2 in class")
                            node.add_child(Node(Id=treeChildren[1], depth=node.depth + 1, parent=node, cluster_label=cluster_child_two))
                            self.temp_n_leafs += 1
                    elif cluster_child_two in sentences_all_classes:
                #        print("\tc2 in class")
                        self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
                        node.add_child(Node(Id=treeChildren[1], depth=node.depth + 1, parent=node, cluster_label=cluster_child_two))
                        self.temp_n_leafs += 1          
                #    else:
                #        print("\tno class for c1 and c2")
            else:
                if len(node.children) == 1 and self.temp_n_leafs < MAX_LEAFS:
                #    print("one child deeper: {}".format(node.children[0]))
                    self._build_tree(node.children[0], current_depth)
                elif len(node.children) == 2 and self.temp_n_leafs < MAX_LEAFS:
                #    print("two childs deeper:")
                #    print(node.children[0])
                #    print(node.children[1])
                    self._build_tree(node.children[0], current_depth)
                    self._build_tree(node.children[1], current_depth)
                #else:
                #    print("no deeper")
            return node

    def _update_leaf_to_root(self, node, abstract_id, class_predict):
        """ Updates node and all its ancestors up to the root with the abstract's id and the predicted class"""
        node.update_node(abstract_id, class_predict)
        if node.parent != None: # Root has no parent
            self._update_leaf_to_root(node.parent, abstract_id, class_predict)
    

    def set_build_tree(self,node):
        """ Builds the tree and sets the variable tree."""  

        # tree with MAX_LEAFS leafs is constructed. 
        # For sklearn add to each leaf its cluster label based on the children in the tree object from sklearn AgglomerativeClustering
        self.leaf_nodes = []
        if self.mode == "sklearn":
            tree = self._build_tree(node) # construct whole tree
            tree = self._get_cluster_labels_for_leafs(tree) # get labels for leafs
            tree = self._cut_nodes_from_leafs(tree) # cut nodes from bottom of the tree until only leafs with a unique cluster_label exist (Number leaves = MAX_LEAFS)
        elif self.mode == "FBE":
            self.temp_n_leafs = 1
            self.maxDepth = 0
            self._get_maxDepth(0, 0)
            depth = 0
            #print("maxDepth: {}".format(self.maxDepth))
            # build tree by level: create first all children for level 1, then level 2... 
            # Prevents that a tree creates children just in one branch and always goes deeper in case of a max number of leavese
            while self.temp_n_leafs < MAX_LEAFS and depth <= self.maxDepth:
            #    print("\n\ndepth: {}, temp_n_leafs: {}".format(depth, self.temp_n_leafs))
                tree = self._build_tree(node, depth)
                depth += 1

        assert isinstance(tree, Node)
        self.tree = tree
        print("Count nodes: {}; leafs: {}".format(self.count_nodes(), self.count_leafs()))


    def _get_maxDepth(self, i, depth):
        """ get max depth of tree"""
        if depth > self.maxDepth:
            self.maxDepth = depth        
        node = self.tree_hierarchy.iloc[i]
        if len(node.children) == 1:
            self._get_maxDepth(node.children[0], depth+1)
        elif len(node.children) == 2:
            self._get_maxDepth(node.children[0], depth+1)
            self._get_maxDepth(node.children[1], depth+1)


        
    def _get_cluster_labels_for_leafs(self, node):
        """ 
            Get's the cluster labels for each leafs using the cluster labels assigned by
            the output of the sklearn agglomerative clustering algorithm.
        """        
        if len(node.children) == 0: #leaf
            cluster_label = self.clusters_predict[node.node_id]
            node.set_clusterLabel(cluster_label)
        else: # no leaf
            self._get_cluster_labels_for_leafs(node.children[0])
            self._get_cluster_labels_for_leafs(node.children[1])
        return node
    
    def _cut_nodes_from_leafs(self, node):
        """ 
            self.mode == sklearn:
            Children of nodes, who are leafs and have the same cluster_label, are cut off
            and the parent node takes the cluster label of its children.
            This is done recursively until there are only leafs with unique cluster_labels 
            Number of leaves = MAX_LEAFS
        
            self.mode == FBE:
            Towards the bottom of the tree, it may happen that a node has only child, which has only one child, 
            and this child also has only one child, etc. Several nodes following of each other with only one child.
            In this case keep only the child C whose parent has two children and cut the child of C.
        """
        if self.mode == "sklearn":
            if len(node.children) > 0: 
                left_child = node.children[0]
                right_child = node.children[1]
                if left_child.cluster_label is None: # left child is not leaf 
                    self._cut_nodes_from_leafs(left_child)
                if right_child.cluster_label is None: # right child is not leaf 
                    self._cut_nodes_from_leafs(right_child)

                # should be updated now
                left_child = node.children[0]
                right_child = node.children[1]
                if left_child.cluster_label == right_child.cluster_label and left_child.cluster_label is not None:
                    node.children = []
                    node.cluster_label = left_child.cluster_label
                    return node
        elif self.mode == "FBE":
            if len(node.children) == 1: # node has only one child
                temp = node
                while len(temp.children) == 1: # check if several nodes following of each other have only one child two
                    temp = temp.children[0]
                if len(temp.children) == 2: # if at some point a node has two children, continue to search
                    self._cut_nodes_from_leafs(temp.children[0])    
                    self._cut_nodes_from_leafs(temp.children[1])
                else: # if we reached a leaf, cut the node's children
                    node.children = []
            elif len(node.children) == 2:
                self._cut_nodes_from_leafs(node.children[0])    
                self._cut_nodes_from_leafs(node.children[1])             
        return node    

    
        
    def fitTree(self, node, data):
        """ Updates all the nodes of the tree according to the clustering from bottom to top """

        assert isinstance(node, Node)
        if len(node.children) > 0: # no leaf
            for child in node.children:
                self.fitTree(child, data)
        else: # leaf
            if self.mode == "sklearn": 
                leaf_cluster_label = node.cluster_label
                abstract_hits = data[data["class_predict"] == leaf_cluster_label]
                for i, row in abstract_hits.iterrows():
                    leaf_abstract_id = row.name
                    leaf_abstract_class_true = row.mesh_ui_diab # true class 
                    self._update_leaf_to_root(node, leaf_abstract_id, leaf_abstract_class_true)
            elif self.mode == "FBE": # several documents per leaf
                leaf_cluster_label = node.cluster_label
                abstract_hits = data[data["uniqueCluster"] == leaf_cluster_label]
                for i, row in abstract_hits.iterrows():
                    leaf_abstract_id = row["id"]
                    leaf_abstract_class_true = row["mesh_ui_diab"]
                    self._update_leaf_to_root(node, leaf_abstract_id, leaf_abstract_class_true)
            else: 
                print("ERROR: mode should be one of ['sklearn', 'FBE']")
        return node
         
            
    def count_nodes(self, tree=None):
        self.n_nodes = 0

        def _walk_count_nodes(node):
            self.n_nodes += 1

            for child in node.children:
                _walk_count_nodes(child)   
                
        if tree == None:
            _walk_count_nodes(self.tree)
        else:
            _walk_count_nodes(tree)
        return self.n_nodes
    

                
    def count_leafs(self, tree=None):

        def _walk_count_leafs(node):
            if node.children == []:
                self.n_leafs += 1
                self.leaf_nodes.append(node)
            else:
                for child in node.children:
                    _walk_count_leafs(child)
        
        self.n_leafs = 0
        self.leaf_nodes = []
        if tree == None:
            _walk_count_leafs(self.tree)
        else:
            _walk_count_leafs(tree)
        return self.n_leafs
    
    
    def get_leaf_nodes(self):
        def _walk_leaf_nodes(node):
            if node.children == []:
                self.leaf_nodes.append(node)
            else:
                for child in node.children:
                    _walk_leaf_nodes(child)
        
        self.leaf_nodes = []
        _walk_leaf_nodes(self.tree)
        return self.leaf_nodes
    
    def _walk_precision(self, node):
        node_precision = node.get_precision()
        self.precision_all_nodes.append(node_precision)
        self.precision_all_nodes_weighted.append(node_precision * node.counts)
        self.precision_all_nodes_weights += node.counts
        for child in node.children:
            self._walk_precision(child)
            
    def get_precision(self):
        self.precision_all_nodes = []
        self.precision_all_nodes_weighted = []
        self.precision_all_nodes_weights = 0
        self._walk_precision(self.tree)
        self.precision_macro = np.mean(self.precision_all_nodes)
        self.precision_micro = np.sum(self.precision_all_nodes_weighted) / self.precision_all_nodes_weights
        return {"prec_macro" : self.precision_macro
                , "prec_micro" : self.precision_micro}

        
    def get_recall(self):
        
        self.recall_all_classes = []
        self.recall_all_classes_weighted = []
        def _walk_recall(node, c):
            """ Get cluster with max documents of class c in which class c is the majority class """
            class_counts = Counter(node.true_classes).most_common()
            majority_classes = [c for c, occ in class_counts  if occ == class_counts[0][1]] # there can be several majority classes in a node
            #majority_class = Counter(node.true_classes).most_common()[0][0]
            occ = node.true_classes.count(c)
            #print()
            #print(node)
            #print("\t{}".format(node.true_classes))
            #print("\tmajority_classe: {}, occ({}): {}".format(majority_classes, c, occ))
            if c in majority_classes and occ > self.temp_max_occ_class_in_cluster:
                self.temp_max_occ_class_in_cluster = occ
            #    print("\t updatetemp_max_occ_class_in_cluster: {}".format(self.temp_max_occ_class_in_cluster))
            #if (occ > self.temp_max_occ_class_in_cluster 
            #    and (c in majority_classes or node.children == [])
            #   ): # if we found a cluster with higher occ of documents for class c and the class c is the majority class in the cluster or leaf node
            #    self.temp_max_occ_class_in_cluster = occ
            #    print("\tupdatetemp_max_occ_class_in_cluster: {}".format(self.temp_max_occ_class_in_cluster))
            
            #if (occ > self.temp_max_occ_class_in_cluster and c in majority_classes):
            # self.temp_max_occ_class_in_cluster = occ
            #    print("\MAJ: tupdatetemp_max_occ_class_in_cluster: {}".format(self.temp_max_occ_class_in_cluster))
            #elif (occ > self.temp_max_occ_class_in_cluster and node.children == []):
            #    self.temp_max_occ_class_in_cluster = occ
            #    print("\tLEAF: updatetemp_max_occ_class_in_cluster: {}".format(self.temp_max_occ_class_in_cluster))
    
            for child in node.children:
                _walk_recall(child, c)
        
        weights_sum = 0
        for c in self.true_classes_documents_unique:
            N_c = self.true_classes_documents.count(c)
            #print("\nc: {}, N_c: {}".format(c, N_c))
            self.temp_max_occ_class_in_cluster = 0
#            _walk_recall(self.tree, c)
            # TODO: check if it is right!
            # # start with children; otherwise recalls for all classes will be highest in root
            _walk_recall(self.tree.children[0], c) 
            _walk_recall(self.tree.children[1], c)
            recall = self.temp_max_occ_class_in_cluster / N_c
            #print("c: {}, recall: {}".format(c, recall))

            self.recall_all_classes.append(recall) #len(self.unique_cluster_predict))
            self.recall_all_classes_weighted.append(recall * N_c)
            weights_sum += N_c
        self.recall_macro = np.mean(self.recall_all_classes)
        self.recall_micro = np.sum(self.recall_all_classes_weighted) / weights_sum
        return {"recall_macro" : self.recall_macro
                ,"recall_micro" : self.recall_micro}
    
    def get_F1(self):
        precision = self.get_precision()
        recall = self.get_recall()        
        
        self.F1_macro = 2*precision["prec_macro"]*recall["recall_macro"] / (precision["prec_macro"] + recall["recall_macro"])
        self.F1_micro = 2*precision["prec_micro"]*recall["recall_micro"] / (precision["prec_micro"] + recall["recall_micro"])
        return {"F1_macro":self.F1_macro
               ,"F1_micro":self.F1_micro}
    
    def F1_zhao(self):
        """ F1 score like in Evaluation of Hierarchical Clustering Algorithms forDocument Datasets from Zhao & Karypis """
        
        def _walk_F1_zhao(node, c, N_c):
            """ Calculates F1 Score for a given class c (N_c = total number of documents of class c) """
            class_count = node.true_classes.count(c)
            prec =  class_count / node.counts
            recall = class_count / N_c
            #print(node)
            #print("\t class_count: {}, self.count: {}, p: {:.2f}, r: {:.2f}".format(class_count, node.counts, prec, recall))
            if prec > 1e-10 or recall > 1e-10: # if prec or recall == 0 
                F1 = 2 * prec * recall / (prec+recall)
            else:
                F1 = 0
            
            if F1 > self.temp_max_doc_perClass_inCluster:
                self.temp_max_doc_perClass_inCluster = F1
                #print("\t\t UPDATE F1: {}!!! ".format(F1))

            for child in node.children:
                _walk_F1_zhao(child, c, N_c)        
        
        FScore_sum = 0
        for c in self.true_classes_documents_unique:
            N_c = self.true_classes_documents.count(c)
            N = len(self.true_classes_documents)
            #print("\nc: {}, N_c: {}, N: {}, N_c/N : {}".format(c, N_c, N, N_c/N))
            self.temp_max_doc_perClass_inCluster = 0
            _walk_F1_zhao(self.tree.children[0], c, N_c) 
            _walk_F1_zhao(self.tree.children[1], c, N_c)
            #print("Best F1: {}".format(self.temp_max_doc_perClass_inCluster))
            #print("Nc/N + F1 : {}".format((N_c / N ) * self.temp_max_doc_perClass_inCluster))
            FScore_sum += (N_c / N ) * self.temp_max_doc_perClass_inCluster
            
        return FScore_sum
    
    def get_isim(self, data):
        """ Internal similarity """
        
        I_sum = 0 
        def _walk_isim(node):
            
            print("Node: {}".format(node))
            print("abstracts: {}".node.abstracts)
            for child in node.children:
                _walk_isim(child)
            

    
    def get_performances(self):
        precision = self.get_precision()
        recall = self.get_recall()
        F1 = self.get_F1()
        return({
            "prec_micro" : precision["prec_micro"]
            ,"prec_macro" : precision["prec_macro"]            
            ,"recall_micro" : recall["recall_micro"]
            ,"recall_macro" : recall["recall_macro"]
            ,"F1_micro" : F1["F1_micro"]
            ,"F1_macro" : F1["F1_macro"]
            ,"F1_zhao" : self.F1_zhao()
        })
 


class Node(object):
    "Generic tree node."
    def __init__(self, Id, depth, parent=None, cluster_label=None, children=[]):
        self.node_id = Id
        self.parent = parent
        self.children = []
        self.depth = depth
        self.cluster_label = cluster_label # In case FBE: this is the filterValue in the leafs
        self.abstracts = [] # PMID's of abstracts 
        self.true_classes = [] # True classes for each abstract
        self.counts = 0
        self.recall = None
        self.precision = None 
        self.F1 = None
        if children is not None:
            for child in children:
                self.add_child(child)
                
    def __repr__(self):
        return "Node id: {} (depth: {}, cluster_label: {}, children: {})".format(self.node_id
                                                                                 , self.depth
                                                                                 , self.cluster_label
                                                                                , [child.node_id for child in self.children])
    
    def add_child(self, node):
        assert isinstance(node, Node)
        self.children.append(node)
        
    def set_clusterLabel(self, clusterLabel):
        self.cluster_label = clusterLabel
        
    def pretty_print(self, depth=0):
        
        if self.depth == depth: 
            print("Node: {}, Parent: {} (Depth: {}, counts: {}, cluster_label: {}) | Children: {}".format(self.node_id, self.parent, self.depth, self.counts, self.cluster_label, self.children))
            print("\tAbstracts: {}".format(Counter(self.abstracts)))
            print("\ttrue_classes: {}".format(Counter(self.true_classes)))
        else:
            for child in self.children:
                child.pretty_print(depth)
            
            
    def update_node(self, abstract_id, true_class):
        """ Updates the abstracts and its true class label running through this node """
        self.abstracts.append(abstract_id)
        self.true_classes.append(true_class)
        self.counts += 1
        
        
    def get_precision(self):
        count = Counter(self.true_classes)
        mostFrequent = max(self.true_classes, key=count.get)
        prec = self.true_classes.count(mostFrequent) / self.counts
        return prec

    def count_class_occurrence(self, c):
        return self.true_classes.count(c)
    

#

In [2]:
data = pd.read_parquet("/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/outputs_03082020/diabetes_abstracts_HC_output.parquet")
data.index = data.index.get_level_values("PMID")
data = data.reset_index()
data["PMID"] = pd.to_numeric(data["PMID"])

#data = pd.read_parquet("/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/diabetes_abstracts_HC_output_10Examples.parquet")
#data = pd.read_parquet("/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/diabetes_abstracts_HC_output_30Examples.parquet")

HC_tree = pd.read_parquet('/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/outputs_03082020/diabetes_abstracts_tree_output.parquet')
#HC_tree = pd.read_parquet('/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/diabetes_abstracts_tree_output_10Examples.parquet')
#HC_tree = pd.read_parquet('/home/adrian/workspace/Hierarchical-Clustering-Active-Learning-Text/diabetes_abstracts_tree_output_30Examples.parquet')

## TEST TREE
#data = pd.DataFrame({"PMID": [0, 1, 2, 3, 4, 5]
#                    , "class_predict": [3, 0, 0, 0, 1, 2]}
#                   , columns=["PMID", "class_predict"]).set_index("PMID")

#HC_tree = pd.DataFrame({"node_id":[6, 7, 8, 9, 10]
#                    , "left" : [1, 2, 0, 5, 8]
#                    , "right" :[3, 6, 4, 7, 9]}
#                   , columns=["node_id", "left", "right"])

print("Tree nodes: {}".format(HC_tree.shape))
print(HC_tree.head())
print("data size: {}".format(data.shape))
print(list(set(data["class_predict"])))
#print(list(set(data["mesh_ui_diab"])))
#print(data["mesh_ui_diab"])

#df_vec = (data.title + " " + data.abstract).map(lambda abstract: avg_feature_vector(abstract))
#print(type(df_vec))
#df_vec = np.stack(df_vec.values, axis = 0)
#print(df_vec.shape)
#print(type(df_vec))

data.head(2)



Tree nodes: (55910, 3)
   node_id   left  right
0    55911  12234  21376
1    55912   2561  48188
2    55913   1107  14060
3    55914  37499  49945
4    55915  20246  51857
data size: (55911, 10)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182

,PMID,title,abstract,date,mesh_ui,mesh_mh,mesh_ui_diab,mesh_mh_diab,title_abstract_prep,class_predict
0,28800712,Outcomes Achieved With Use of a Prefabricated ...,BACKGROUND\nThe total contact cast (TCC) is co...,2017-10,"D000328,D000367,D000368,D000369,D002370,D01533...","Adult,Age Factors,Aged,Aged, 80 and over,Casts...",D017719,Diabetic Foot,outcomes achieved with use of a prefabricated ...,34
1,6989594,Investigation of insulin sensitivity in early ...,Twenty-three normal weight subjects without an...,1980-01,"D001786,D001835,D005230,D005951,D006801,D00732...","Blood Glucose,Body Weight,Fatty Acids, Noneste...",D011236,Prediabetic State,investigation of insulin sensitivity in early ...,34


In [94]:
# ADD vectors to abstracts

#model = KeyedVectors.load_word2vec_format("/home/adrian/PhD/Data/Word2Vec/BioASQvectors2018/pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin", binary=True)
#index2word_set = set(model.wv.index2word)
# clean for BioASQ
#bioclean = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'",'').strip().lower()).split()

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def avg_feature_vector(sentence, model=model, num_features=200, index2word_set=index2word_set):
    #words = sentence.split()
    try:
        words = bioclean(sentence)
    except:
        print("bioclean did not work for: {}".format(sentence))
        print(type(sentence))
        print(math.isnan(sentence))
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
        else:
            if hasNumbers(word):
                print("word not in vocabulary: {}".format(word))
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

#data["vec"] = (data.title + " " + data.abstract).map(lambda abstract: avg_feature_vector(abstract))
#data.head()

In [3]:
# initialise
#MAX_LEAFS=8
treeClass = Tree(HC_tree, data["class_predict"], mode="sklearn", true_classes_all=data["mesh_ui_diab"])

# define root node
root = Node(Id=HC_tree["node_id"].max() # In scikit learn, the root node is the one with maximum node id
          , depth=0
          , parent=None
          , children=[])

# build tree
treeClass.set_build_tree(root)

print("N nodes: {}".format(treeClass.count_nodes()))
print("N leafs: {}".format(treeClass.count_leafs()))

treeClass.leaf_nodes

Count nodes: 511; leafs: 256
N nodes: 511
N leafs: 256


[Node id: 99614 (depth: 1, cluster_label: 132, children: []),
 Node id: 9920 (depth: 3, cluster_label: 143, children: []),
 Node id: 90245 (depth: 3, cluster_label: 123, children: []),
 Node id: 111266 (depth: 3, cluster_label: 68, children: []),
 Node id: 42262 (depth: 4, cluster_label: 163, children: []),
 Node id: 110858 (depth: 6, cluster_label: 165, children: []),
 Node id: 598 (depth: 7, cluster_label: 146, children: []),
 Node id: 54801 (depth: 7, cluster_label: 126, children: []),
 Node id: 8842 (depth: 6, cluster_label: 223, children: []),
 Node id: 110904 (depth: 7, cluster_label: 148, children: []),
 Node id: 48600 (depth: 9, cluster_label: 184, children: []),
 Node id: 53994 (depth: 9, cluster_label: 177, children: []),
 Node id: 5257 (depth: 11, cluster_label: 135, children: []),
 Node id: 110592 (depth: 12, cluster_label: 218, children: []),
 Node id: 23650 (depth: 13, cluster_label: 200, children: []),
 Node id: 39586 (depth: 13, cluster_label: 149, children: []),
 Node 

In [4]:
# fit tree with abstracts 
tree_fit = treeClass.fitTree(treeClass.tree, data)

In [5]:
#treeClass.tree.pretty_print(depth=8)

In [6]:
#print(treeClass.get_precision())
#print(treeClass.get_recall())
#print(treeClass.get_F1())
pprint(treeClass.get_performances())
print("F1 score zhao: {}".format(treeClass.F1_zhao()))


{'F1_macro': 0.4441254035533422,
 'F1_micro': 0.19230689421131783,
 'F1_zhao': 0.23525969824232465,
 'prec_macro': 0.5810909145747698,
 'prec_micro': 0.11243403384883459,
 'recall_macro': 0.3594108035763631,
 'recall_micro': 0.6640374881508111}
F1 score zhao: 0.23525969824232465


# load FeedbackExplorer output

In [7]:
import pyspark
#fbe_path = "/home/adrian/tmp/Test_FBE"
fbe_path = "/home/adrian/workspace/FBE output/maxTopwords_6_maxClasses1024_Nall"

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sentences = spark.read.load(fbe_path+"/phrases/")
print(len(sentences.columns))
print(sentences.count())

df_short = sentences.select("id", "tokens", "index")
#df_short.printSchema()
df_short.show(2)

1027
55911
+--------+--------------------+--------------------+
|      id|              tokens|               index|
+--------+--------------------+--------------------+
|28800712|[outcomes,  , ach...|[191 -> [218 -> 2...|
| 6989594|[investigation,  ...|[107 -> [92 -> 92...|
+--------+--------------------+--------------------+
only showing top 2 rows



In [8]:
nodes = pd.read_json(fbe_path+"/nodes.json", orient="records")
print(nodes.shape)
nodes.head(5)

(512, 21)


,name,tagId,color,annotations,algo,strLinks,strClassPath,names,filterMode,filterValue,...,windowSize,classCenters,cError,childSplitSize,children,hits,metrics,rocCurve,externalClassesFreq,purity
0,In Scope,0.0,NaN,"[{'tokens': ['aggregate'], 'tag': 1, 'from': N...",{'value': 'supervised'},{'0': [1]},{'1': [0]},{},{'value': 'allIn'},[0],...,0.0,None,None,NaN,[1],55911,{},{},{},{}
1,Explorer,1.0,NaN,"[{'tokens': ['diabetic'], 'tag': 2, 'from': No...",{'value': 'clustering'},"{'1': [2, 3]}","{'2': [0, 1], '3': [0, 1]}",{},{'value': 'anyIn'},[1],...,NaN,"{'2': 0, '3': 1}","[NaN, NaN]",50.0,"[2, 187]",55911,{},{},{},{}
2,Explorer,NaN,NaN,"[{'tokens': ['patients'], 'tag': 4, 'from': No...",{'value': 'clustering'},"{'1': [4, 5]}","{'4': [0, 1, 2], '5': [0, 1, 2]}",{},{'value': 'anyIn'},[2],...,NaN,"{'4': 0, '5': 1}","[NaN, NaN]",50.0,"[3, 88]",25654,{},{},{},{}
3,Explorer,NaN,NaN,"[{'tokens': ['extragenital'], 'tag': 32, 'from...",{'value': 'clustering'},"{'1': [32, 33]}","{'32': [0, 1, 2, 4], '33': [0, 1, 2, 4]}",{},{'value': 'anyIn'},[4],...,NaN,"{'32': 0, '33': 1}","[NaN, NaN]",50.0,"[4, 29]",20825,{},{},{},{}
4,Explorer,NaN,NaN,"[{'tokens': ['modifable'], 'tag': 80, 'from': ...",{'value': 'clustering'},"{'1': [80, 81]}","{'80': [0, 1, 2, 32, 4], '81': [0, 1, 2, 32, 4]}",{},{'value': 'anyIn'},[32],...,NaN,"{'80': 0, '81': 1}","[NaN, NaN]",50.0,"[5, 24]",1582,{},{},{},{}


In [9]:
# Get list with all possible classes in the sentences file
import pyspark
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *


def get_list_all_possible_classes(sentences ):
    """ Get the list of all possible occuring classes in the sentences file """
    join_udf = udf(lambda x: ";".join(x))
    sentences_classes_udf = udf(lambda x: ";".join([str(v) for v in x.keys()]))

    sentences_transformed = sentences.select("id"
                                            , "tokens"
                                            , sentences_classes_udf('index').alias("all_classes")) \
                                    .withColumn("tokens", join_udf(col("tokens"))) 
                    
    sentences_pdf = sentences_transformed.toPandas()
    sentences_pdf["id"] = pd.to_numeric(sentences_pdf["id"])

    # add true class labels to sentences from data by merge/join 
    sentences_pdf["PMID"] = sentences_pdf["id"]
    sentences_pdf["PMID"] = pd.to_numeric(sentences_pdf["PMID"])
    meshDiab = data[["PMID", "mesh_ui_diab"]]
    meshDiab["PMID"] = pd.to_numeric(meshDiab["PMID"])
    sentences_pd_with_classes = pd.merge(sentences_pdf, meshDiab, on='PMID', how="left")

    print("\tsentences_pdf: {}".format(sentences_pdf.shape))
    print("\tmeshDiab: {}".format(meshDiab.shape))
    print("\tmerged: {}".format(sentences_pd_with_classes.shape))

    
    # list of all classes in the sentences file
    return (set(pd.to_numeric(sentences_pdf["all_classes"].map(lambda sentence: sentence.split(";")).explode()).values)
            , sentences_pd_with_classes)

sentences_all_classes, sentences_pd_with_classes = get_list_all_possible_classes(sentences)
print("Number of classes in sentences file: {}".format(len(sentences_all_classes)))
print("Merged dataset with true classes: {}".format(sentences_pd_with_classes.shape))
sentences_pd_with_classes.head()

/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


	sentences_pdf: (55911, 4)
	meshDiab: (55911, 2)
	merged: (55911, 5)
Number of classes in sentences file: 913
Merged dataset with true classes: (55911, 5)


,id,tokens,all_classes,PMID,mesh_ui_diab
0,28800712,outcomes; ;achieved; ;with; ;use; ;of; ;a; ;pr...,0;1;67;195;3;7;10;13;847;850;723;27;61;30;191,28800712,D017719
1,6989594,investigation; ;of; ;insulin; ;sensitivity; ;i...,0;1;3;103;7;10;107;27;13;31;63,6989594,D011236
2,524360,ultrastructural; ;pathology; ;of; ;peripheral;...,0;1;33;2;83;595;4;935;91;157;94,524360,D003929
3,21199315,evidence;-;based; ;interventional; ;pain; ;med...,0;1;994;3;70;7;328;712;10;12;724;917;25,21199315,D003929
4,24607755,delivery; ;timing; ;and; ;cesarean; ;delivery;...,0;1;33;2;82;4;709;214;89;730;252,24607755,D016640


In [10]:
# initialise
treeFBE = Tree(nodes
            #, list(set(data["class_predict"]))
            , mode="FBE"
            , sentences_all_classes=sentences_all_classes
            , true_classes_all=sentences_pd_with_classes["mesh_ui_diab"])

# define root node
root = Node(Id=1, depth=0, parent=None, children=[]) # Id = 1 because start at Explorer 

# build tree
#maxDepth = 10
treeFBE.set_build_tree(root)

print("Number leafs: {}".format(treeFBE.count_leafs()))

Count nodes: 484; leafs: 236
Number leafs: 236


In [12]:
# Associate cluster to each sentence

from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

def matchCluster(index_map, cluster): 
    """ gets for each abstract its unique cluster (filterValue) from the index"""
    return list(set(list(index_map.keys())).intersection(set(cluster)))[0]

def associate_unique_cluster_to_documents(sentences, tree):
    """ Associates unique cluster to each document """
    leafs = tree.get_leaf_nodes()
    print("N leafs: {}".format(len(leafs)))
    cluster = [leaf.cluster_label for leaf in leafs]
    print("N clusters: {}".format(len(set(cluster))))

    matchCluster_udf = udf(lambda y: matchCluster(y, cluster))
    join_udf = udf(lambda x: ";".join(x))

    sentences_transformed = sentences.select("id", "tokens", matchCluster_udf('index').alias("uniqueCluster")) \
                        .withColumn("tokens", join_udf(col("tokens"))) 

    #sentences.select('index', matchClass_udf('index').atlias("uniqueCluster")).groupby("uniqueCluster").count().show()
    sentences_pd = sentences_transformed.toPandas()
    sentences_pd["id"] = pd.to_numeric(sentences_pd["id"])
    sentences_pd["uniqueCluster"] = pd.to_numeric(sentences_pd["uniqueCluster"])
    
    # add true class labels to data by merge/join 
    sentences_pd["PMID"] = sentences_pd["id"]
    sentences_pd["PMID"] = pd.to_numeric(sentences_pd["PMID"])
    meshDiab = data[["PMID", "mesh_ui_diab"]]
    meshDiab["PMID"] = pd.to_numeric(meshDiab["PMID"])
    sentences_pd_with_classes_uniqueCluster = pd.merge(sentences_pd, meshDiab, on='PMID', how="left")
    print("sentences_pd: {}".format(sentences_pd.shape))
    print("meshDiab: {}".format(meshDiab.shape))
    print("sentences_pd_with_classes_uniqueCluster: {}".format(sentences_pd_with_classes.shape))
    
    return sentences_pd_with_classes_uniqueCluster 

sentences_pd_with_classes_uniqueCluster = associate_unique_cluster_to_documents(sentences, treeFBE)
print("Unique clusters in sentences: {}".format(sentences_pd_with_classes_uniqueCluster["uniqueCluster"].nunique())) #####

sentences_pd_with_classes_uniqueCluster.head()

N leafs: 236
N clusters: 236
sentences_pd: (55911, 4)
meshDiab: (55911, 2)
sentences_pd_with_classes_uniqueCluster: (55911, 5)
Unique clusters in sentences: 236


/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,tokens,uniqueCluster,PMID,mesh_ui_diab
0,28800712,outcomes; ;achieved; ;with; ;use; ;of; ;a; ;pr...,847,28800712,D017719
1,6989594,investigation; ;of; ;insulin; ;sensitivity; ;i...,103,6989594,D011236
2,524360,ultrastructural; ;pathology; ;of; ;peripheral;...,595,524360,D003929
3,21199315,evidence;-;based; ;interventional; ;pain; ;med...,917,21199315,D003929
4,24607755,delivery; ;timing; ;and; ;cesarean; ;delivery;...,709,24607755,D016640


In [13]:
treeFBE.fitTree(treeFBE.tree, sentences_pd_with_classes_uniqueCluster)

leaf_abstract_id: 2648659, leaf_abstract_class_true: D003928
leaf_abstract_id: 8588876, leaf_abstract_class_true: D003930
leaf_abstract_id: 8164551, leaf_abstract_class_true: D016640
leaf_abstract_id: 21816498, leaf_abstract_class_true: D016640
leaf_abstract_id: 10827618, leaf_abstract_class_true: D003924
leaf_abstract_id: 17487113, leaf_abstract_class_true: D016640
leaf_abstract_id: 22101483, leaf_abstract_class_true: D017719
leaf_abstract_id: 26756260, leaf_abstract_class_true: D016640
leaf_abstract_id: 16522427, leaf_abstract_class_true: D016640
leaf_abstract_id: 28400430, leaf_abstract_class_true: D011236
leaf_abstract_id: 21441441, leaf_abstract_class_true: D003924
leaf_abstract_id: 30226486, leaf_abstract_class_true: D003920
leaf_abstract_id: 20204997, leaf_abstract_class_true: D003920
leaf_abstract_id: 11787436, leaf_abstract_class_true: D003928
leaf_abstract_id: 24620558, leaf_abstract_class_true: D005320
leaf_abstract_id: 25757637, leaf_abstract_class_true: D016640
leaf_abstra

leaf_abstract_id: 25660138, leaf_abstract_class_true: D003925
leaf_abstract_id: 2078874, leaf_abstract_class_true: D003922
leaf_abstract_id: 16098925, leaf_abstract_class_true: D016640
leaf_abstract_id: 15172859, leaf_abstract_class_true: D016640
leaf_abstract_id: 27227919, leaf_abstract_class_true: D003924
leaf_abstract_id: 30260983, leaf_abstract_class_true: D003920
leaf_abstract_id: 22926361, leaf_abstract_class_true: D016640
leaf_abstract_id: 24146865, leaf_abstract_class_true: D003930
leaf_abstract_id: 29203507, leaf_abstract_class_true: D016640
leaf_abstract_id: 3984732, leaf_abstract_class_true: D003922
leaf_abstract_id: 23080236, leaf_abstract_class_true: D017719
leaf_abstract_id: 30500542, leaf_abstract_class_true: D016640
leaf_abstract_id: 15735191, leaf_abstract_class_true: D016640
leaf_abstract_id: 16317371, leaf_abstract_class_true: D003930
leaf_abstract_id: 17934394, leaf_abstract_class_true: D016640
leaf_abstract_id: 27601018, leaf_abstract_class_true: D016640
leaf_abstr

leaf_abstract_id: 2951460, leaf_abstract_class_true: D003925
leaf_abstract_id: 3883000, leaf_abstract_class_true: D048909
leaf_abstract_id: 15785022, leaf_abstract_class_true: D003925
leaf_abstract_id: 19347224, leaf_abstract_class_true: D005320
leaf_abstract_id: 9255040, leaf_abstract_class_true: D016640
leaf_abstract_id: 25082340, leaf_abstract_class_true: D017719
leaf_abstract_id: 21925081, leaf_abstract_class_true: D003920
leaf_abstract_id: 25870695, leaf_abstract_class_true: D016640
leaf_abstract_id: 21957592, leaf_abstract_class_true: D003929
leaf_abstract_id: 26813594, leaf_abstract_class_true: D048909
leaf_abstract_id: 12694990, leaf_abstract_class_true: D003929
leaf_abstract_id: 1456792, leaf_abstract_class_true: D003928
leaf_abstract_id: 27004003, leaf_abstract_class_true: D048909
leaf_abstract_id: 27924661, leaf_abstract_class_true: D016640
leaf_abstract_id: 8865924, leaf_abstract_class_true: D003928
leaf_abstract_id: 26048000, leaf_abstract_class_true: D048909
leaf_abstract

leaf_abstract_id: 20222331, leaf_abstract_class_true: D048909
leaf_abstract_id: 10395336, leaf_abstract_class_true: D017719
leaf_abstract_id: 29424871, leaf_abstract_class_true: D005320
leaf_abstract_id: 30805100, leaf_abstract_class_true: D016640
leaf_abstract_id: 19484867, leaf_abstract_class_true: D003928
leaf_abstract_id: 10960718, leaf_abstract_class_true: D017719
leaf_abstract_id: 12076235, leaf_abstract_class_true: D048909
leaf_abstract_id: 24642738, leaf_abstract_class_true: D017719
leaf_abstract_id: 7198388, leaf_abstract_class_true: D003930
leaf_abstract_id: 20697004, leaf_abstract_class_true: D048909
leaf_abstract_id: 30385355, leaf_abstract_class_true: D005320
leaf_abstract_id: 30630990, leaf_abstract_class_true: D003920
leaf_abstract_id: 25016791, leaf_abstract_class_true: D003930
leaf_abstract_id: 25672881, leaf_abstract_class_true: D003920
leaf_abstract_id: 18351194, leaf_abstract_class_true: D003920
leaf_abstract_id: 31471505, leaf_abstract_class_true: D003920
leaf_abst

leaf_abstract_id: 29636823, leaf_abstract_class_true: D017719
leaf_abstract_id: 24564044, leaf_abstract_class_true: D003920
leaf_abstract_id: 18386800, leaf_abstract_class_true: D005320
leaf_abstract_id: 17426042, leaf_abstract_class_true: D003922
leaf_abstract_id: 23781894, leaf_abstract_class_true: D003924
leaf_abstract_id: 28764665, leaf_abstract_class_true: D016640
leaf_abstract_id: 19390341, leaf_abstract_class_true: D048909
leaf_abstract_id: 20423440, leaf_abstract_class_true: D003922
leaf_abstract_id: 9638541, leaf_abstract_class_true: D017719
leaf_abstract_id: 28452722, leaf_abstract_class_true: D005320
leaf_abstract_id: 22534170, leaf_abstract_class_true: D003924
leaf_abstract_id: 507535, leaf_abstract_class_true: D048909
leaf_abstract_id: 22142620, leaf_abstract_class_true: D003925
leaf_abstract_id: 31045693, leaf_abstract_class_true: D003920
leaf_abstract_id: 1196554, leaf_abstract_class_true: D048909
leaf_abstract_id: 7814784, leaf_abstract_class_true: D048909
leaf_abstract

...

leaf_abstract_id: 11192103, leaf_abstract_class_true: D016640
leaf_abstract_id: 23207871, leaf_abstract_class_true: D003921
leaf_abstract_id: 15838400, leaf_abstract_class_true: D006944
leaf_abstract_id: 24124964, leaf_abstract_class_true: D003924
leaf_abstract_id: 24964071, leaf_abstract_class_true: D003920
leaf_abstract_id: 26769102, leaf_abstract_class_true: D016640
leaf_abstract_id: 21109476, leaf_abstract_class_true: D016640
leaf_abstract_id: 18437353, leaf_abstract_class_true: D016640
leaf_abstract_id: 3534980, leaf_abstract_class_true: D003922
leaf_abstract_id: 24746173, leaf_abstract_class_true: D003922
leaf_abstract_id: 16967811, leaf_abstract_class_true: D003929
leaf_abstract_id: 1872305, leaf_abstract_class_true: D003925
leaf_abstract_id: 9230065, leaf_abstract_class_true: D048909
leaf_abstract_id: 27390971, leaf_abstract_class_true: D003924
leaf_abstract_id: 11793023, leaf_abstract_class_true: D003924
leaf_abstract_id: 10641958, leaf_abstract_class_true: D003924
leaf_abstra

leaf_abstract_id: 24389556, leaf_abstract_class_true: D003921
leaf_abstract_id: 27465377, leaf_abstract_class_true: D016640
leaf_abstract_id: 14746166, leaf_abstract_class_true: D003925
leaf_abstract_id: 25450302, leaf_abstract_class_true: D016640
leaf_abstract_id: 12830021, leaf_abstract_class_true: D003924
leaf_abstract_id: 23470316, leaf_abstract_class_true: D016640
leaf_abstract_id: 19625702, leaf_abstract_class_true: D003924
leaf_abstract_id: 11480453, leaf_abstract_class_true: D003925
leaf_abstract_id: 3930307, leaf_abstract_class_true: D016883
leaf_abstract_id: 23520452, leaf_abstract_class_true: D003924
leaf_abstract_id: 3082174, leaf_abstract_class_true: D003924
leaf_abstract_id: 30496135, leaf_abstract_class_true: D011236
leaf_abstract_id: 29461236, leaf_abstract_class_true: D003924
leaf_abstract_id: 22326749, leaf_abstract_class_true: D003920
leaf_abstract_id: 28401628, leaf_abstract_class_true: D003922
leaf_abstract_id: 6672992, leaf_abstract_class_true: D003922
leaf_abstra

leaf_abstract_id: 15133797, leaf_abstract_class_true: D005320
leaf_abstract_id: 19636976, leaf_abstract_class_true: D016640
leaf_abstract_id: 3041155, leaf_abstract_class_true: D003930
leaf_abstract_id: 11349170, leaf_abstract_class_true: D016640
leaf_abstract_id: 8099822, leaf_abstract_class_true: D005320
leaf_abstract_id: 22730196, leaf_abstract_class_true: D017719
leaf_abstract_id: 21295850, leaf_abstract_class_true: D016640
leaf_abstract_id: 4000644, leaf_abstract_class_true: D003930
leaf_abstract_id: 12507626, leaf_abstract_class_true: D003930
leaf_abstract_id: 28110940, leaf_abstract_class_true: D048909
leaf_abstract_id: 412058, leaf_abstract_class_true: D048909
leaf_abstract_id: 2976761, leaf_abstract_class_true: D003930
leaf_abstract_id: 25537242, leaf_abstract_class_true: D005320
leaf_abstract_id: 15902122, leaf_abstract_class_true: D005320
leaf_abstract_id: 2971076, leaf_abstract_class_true: D003930
leaf_abstract_id: 26019142, leaf_abstract_class_true: D003925
leaf_abstract_i

leaf_abstract_id: 20070983, leaf_abstract_class_true: D003925
leaf_abstract_id: 10584509, leaf_abstract_class_true: D003922
leaf_abstract_id: 19027331, leaf_abstract_class_true: D003922
leaf_abstract_id: 11193214, leaf_abstract_class_true: D005320
leaf_abstract_id: 28926293, leaf_abstract_class_true: D048909
leaf_abstract_id: 27738926, leaf_abstract_class_true: D003921
leaf_abstract_id: 29088414, leaf_abstract_class_true: D003924
leaf_abstract_id: 20216492, leaf_abstract_class_true: D017719
leaf_abstract_id: 29142171, leaf_abstract_class_true: D003920
leaf_abstract_id: 23594738, leaf_abstract_class_true: D005320
leaf_abstract_id: 22380687, leaf_abstract_class_true: D017719
leaf_abstract_id: 23682543, leaf_abstract_class_true: D017719
leaf_abstract_id: 23972010, leaf_abstract_class_true: D048909
leaf_abstract_id: 11960290, leaf_abstract_class_true: D003922
leaf_abstract_id: 22306653, leaf_abstract_class_true: D005320
leaf_abstract_id: 24024338, leaf_abstract_class_true: D003928
leaf_abs

Node id: 1 (depth: 0, cluster_label: None, children: [2, 187])

In [20]:
#print(treeFBE.get_precision_macro())
#print(treeFBE.get_recall_macro())
#print(treeFBE.get_F1())
print(treeFBE.get_performances())

{'prec_micro': 0.2211752045099421, 'prec_macro': 0.3180116921911039, 'recall_micro': 0.28332528482767255, 'recall_macro': 0.16608015672409682, 'F1_micro': 0.24842206950829182, 'F1_macro': 0.21820417673855752}


In [14]:
from pprint import pprint
print("Performance FBE:")
pprint(treeFBE.get_performances())
print()
print("Performance scikit learn: ")
pprint(treeClass.get_performances())

Performance FBE:
{'F1_macro': 0.22118151699607877,
 'F1_micro': 0.2333164135121727,
 'F1_zhao': 0.20859647776081247,
 'prec_macro': 0.3338511201744625,
 'prec_micro': 0.1900367478940206,
 'recall_macro': 0.16537126576469596,
 'recall_micro': 0.30212301693763305}

Performance scikit learn: 
{'F1_macro': 0.4441254035533422,
 'F1_micro': 0.19230689421131783,
 'F1_zhao': 0.23525969824232465,
 'prec_macro': 0.5810909145747698,
 'prec_micro': 0.11243403384883459,
 'recall_macro': 0.3594108035763631,
 'recall_micro': 0.6640374881508111}


In [106]:
sentences.select('index', matchClass_udf('index').alias("uniqueCluster")).groupby("uniqueCluster").count().show()


+-----------+-----+
|uniqueClass|count|
+-----------+-----+
|       1159|   21|
|       1090|  234|
|        296|   51|
|        691|   33|
|        125|    3|
|        666|  256|
|       1280|  334|
|        124| 1199|
|        718|  312|
|        740| 1173|
|        169|   41|
|        747|   46|
|       1425|   19|
|        577|    5|
|        272|   25|
|         54|  968|
|        282|    7|
|        232|    1|
|        483|   27|
|       1158|    5|
+-----------+-----+
only showing top 20 rows



### Calculate Performance for different FBE configurations. 
Limit number of abstracts to 2000; top words: 5

In [1]:
class Tree(object):
    
    def __init__(self, tree_hierarchy, clusters_predict=[], mode="sklearn", sentences_all_classes=None, true_classes_all=None):
        """
        @param mode : Two possible values
            - "FBE" : Tree object for Feedback Explorer output
            - "sklearn" : Tree object for scikit learn output
        
        @param sentences_all_classes : List of all possible classes occuring in the sentences file (only for mode FBE)
        @param true_labels_all : All occuring true labels (mesh codes) of the documents/abstracts
        """
        self.tree = None
        if mode in ["sklearn", "FBE"]:
            self.mode = mode
        else:
            raise ValueError("Provided mode '{}' is not supported".format(mode))
        self.tree_hierarchy = tree_hierarchy # pandas dataframe with tree structure coming from hierarchical clustering
        self.n_nodes = 0 # updated by calling self.count_nodes()
        self.n_leafs = 0 # updated by calling self.count_leafs()
        self.temp_n_leafs = 1 # In mode 'FBE' helps to construct the tree with the right number of nodes
        self.clusters_predict = clusters_predict # predicted cluster for each document
        self.unique_cluster_predict = list(set(clusters_predict)) # list of all classes to calculate performance metrices
        self.leaf_nodes = [] # list of all leaf nodes
        self.sentences_all_classes = sentences_all_classes # List of all classes occuring in sentences (phrases.parquet)
        self.true_classes_documents = true_classes_all.values.tolist() # list of true labels (mesh codes) in the abstracts
        self.true_classes_documents_unique = list(set(true_classes_all)) # all possible occuring true labels (mesh codes) in the abstracts
        self.precision_all_nodes = [] # macro
        self.precision_all_nodes_weighted = []
        self.precision_all_nodes_weights = 0
        self.precision_macro = None 
        self.precision_micro = None
        self.recall_all_classes = []
        self.recall_all_classes_weighted = []
        self.recall_macro = None
        self.recall_micro = None
        self.F1_macro = None
        self.F1_micro = None
        self.maxDepth = 0
        
        
    def _build_tree(self, node, current_depth=None):

        if self.mode == "FBE": 
            # Only create node if node is in current depth level
            if node.depth == current_depth and self.temp_n_leafs < MAX_LEAFS:
                treeChildren = self.tree_hierarchy.iloc[node.node_id].children
                print()
                print("{}".format(node))
                print("tree children: {}".format(treeChildren))
    
                # FBE tree is not a perfect binary tree, some nodes don't create children any more
                if len(treeChildren) > 0:
                    cluster_child_one = self.tree_hierarchy.iloc[treeChildren[0]].filterValue[0]
                    cluster_child_two = self.tree_hierarchy.iloc[treeChildren[1]].filterValue[0]
                    print("\tcluster c1: {}, cluster c2: {}".format(cluster_child_one, cluster_child_two))
                    # Some nodes from nodes.json are empty: no sentences is going through them
                    # Only create node in tree when there is a sentence running through it
#                    if cluster_child_one in self.sentences_all_classes and cluster_child_two in self.sentences_all_classes:
#                        self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
#                        print("\tc1 and c2 have sentences")
#                        node.add_child(Node(Id=treeChildren[0], depth=node.depth + 1, parent=node, cluster_label=cluster_child_one))
#                        node.add_child(Node(Id=treeChildren[1], depth=node.depth + 1, parent=node, cluster_label=cluster_child_two))
#                        self.temp_n_leafs += 2                    
                    if cluster_child_one in self.sentences_all_classes:
                        self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
                        node.add_child(Node(Id=treeChildren[0], depth=node.depth + 1, parent=node, cluster_label=cluster_child_one))
                        self.temp_n_leafs += 1
                        if cluster_child_two in sentences_all_classes:
                            print("\tc1 and c2 have sentences")
                            node.add_child(Node(Id=treeChildren[1], depth=node.depth + 1, parent=node, cluster_label=cluster_child_two))
                            self.temp_n_leafs += 1
                        else:
                            print("\tonly c1 has sentence")
                    elif cluster_child_two in sentences_all_classes:
                        print("\tonly c2 has sentences")
                        self.temp_n_leafs -= 1 # lose one leaf because it is split into two new leafs
                        node.add_child(Node(Id=treeChildren[1], depth=node.depth + 1, parent=node, cluster_label=cluster_child_two))
                        self.temp_n_leafs += 1          
                    else:
                        print("\tno sentence for c1 and c2")
            else:
                if len(node.children) == 1 and self.temp_n_leafs < MAX_LEAFS:
                    print("one child deeper (node.id={}, node depth={})".format(node.node_id, node.depth))
                    self._build_tree(node.children[0], current_depth)
                elif len(node.children) == 2 and self.temp_n_leafs < MAX_LEAFS:
                    print("two childs deeper (node.id={}, node depth : {}):".format(node.node_id, node.depth))
                    print("\t{}".format(node.children[0]))
                    print("\t{}".format(node.children[1]))
                    self._build_tree(node.children[0], current_depth)
                    self._build_tree(node.children[1], current_depth)
                #else:
                #    print("no deeper")
            return node

    def _update_leaf_to_root(self, node, abstract_id, class_predict):
        """ Updates node and all its ancestors up to the root with the abstract's id and the predicted class"""
        node.update_node(abstract_id, class_predict)
        if node.parent != None: # Root has no parent
            self._update_leaf_to_root(node.parent, abstract_id, class_predict)
    

    def set_build_tree(self,node):
        """ Builds the tree and sets the variable tree."""  

        # tree with MAX_LEAFS leafs is constructed. 
        # For sklearn add to each leaf its cluster label based on the children in the tree object from sklearn AgglomerativeClustering
        self.leaf_nodes = []
        if self.mode == "sklearn":
            tree = self._build_tree(node) # construct whole tree
            tree = self._get_cluster_labels_for_leafs(tree) # get labels for leafs
            tree = self._cut_nodes_from_leafs(tree) # cut nodes from bottom of the tree until only leafs with a unique cluster_label exist (Number leaves = MAX_LEAFS)
        elif self.mode == "FBE":
            self.temp_n_leafs = 1
            self.maxDepth = 0
            self._get_maxDepth(0, 0)
            depth = 0
            print("maxDepth: {}".format(self.maxDepth))
            # build tree by level: create first all children for level 1, then level 2... 
            # Prevents that a tree creates children just in one branch and always goes deeper in case of a max number of leavese
            while self.temp_n_leafs < MAX_LEAFS and depth <= 15:#self.maxDepth:
                print("\n\ndepth: {}, temp_n_leafs: {}".format(depth, self.temp_n_leafs))
                tree = self._build_tree(node, depth)
                depth += 1

            self.tree=tree
            print("\n\n\n==========\n")
            print("Count nodes: {}; leafs: {}".format(self.count_nodes(), self.count_leafs()))
            self._cut_nodes_from_leafs(self.tree)
            print("Count nodes: {}; leafs: {}".format(self.count_nodes(), self.count_leafs()))

#        assert isinstance(tree, Node)
#        self.tree = tree
#        print("Count nodes: {}; leafs: {}".format(self.count_nodes(), self.count_leafs()))
        
        

    def _get_maxDepth(self, i, depth):
        """ get max depth of tree"""
        if depth > self.maxDepth:
            self.maxDepth = depth        
        node = self.tree_hierarchy.iloc[i]
        #print(node)
        if len(node.children) == 1:
            self._get_maxDepth(node.children[0], depth+1)
        elif len(node.children) == 2:
            self._get_maxDepth(node.children[0], depth+1)
            self._get_maxDepth(node.children[1], depth+1)
            
    def count_nodes(self, tree=None):
        self.n_nodes = 0

                
        def _walk_count_nodes(node):
            self.n_nodes += 1

            for child in node.children:
                _walk_count_nodes(child)   
                
        if tree == None:
            _walk_count_nodes(self.tree)
        else:
            _walk_count_nodes(tree)
        return self.n_nodes        
    
    def count_leafs(self, tree=None):

        def _walk_count_leafs(node):
            if node.children == []:
                self.n_leafs += 1
                self.leaf_nodes.append(node)
            else:
                for child in node.children:
                    _walk_count_leafs(child)
        
        self.n_leafs = 0
        self.leaf_nodes = []
        if tree == None:
            _walk_count_leafs(self.tree)
        else:
            _walk_count_leafs(tree)
        return self.n_leafs 

    def get_leaf_nodes(self):
        def _walk_leaf_nodes(node):
            if node.children == []:
                self.leaf_nodes.append(node)
            else:
                for child in node.children:
                    _walk_leaf_nodes(child)
        
        self.leaf_nodes = []
        _walk_leaf_nodes(self.tree)
        return self.leaf_nodes    
    
    def _cut_nodes_from_leafs(self, node):
        """ 
            Children of nodes, who are leafs and have the same cluster_label, are cut off
            and the parent node takes the cluster label of its children.
            This is done recursively until there are only leafs with unique cluster_labels 
            Number of leaves = MAX_LEAFS
        
        """
        
        if len(node.children) == 1: 
            print(node)
            temp = node
            while len(temp.children) == 1:
                temp = temp.children[0]
                print("\tchild: {}".format(temp))
            if len(temp.children) == 2:
                print("\t two children")
                self._cut_nodes_from_leafs(temp.children[0])    
                self._cut_nodes_from_leafs(temp.children[1])
            else:
                print("\treset node")
                node.children = []
            #return node
        elif len(node.children) == 2:
            self._cut_nodes_from_leafs(node.children[0])    
            self._cut_nodes_from_leafs(node.children[1])    
        #else:
        #    return node
        return node
#        if len(node.children) > 0: 
#            left_child = node.children[0]
#            right_child = node.children[1]
#            if left_child.cluster_label is None: # left child is not leaf 
#                self._cut_nodes_from_leafs(left_child)
#            if right_child.cluster_label is None: # right child is not leaf 
#                self._cut_nodes_from_leafs(right_child)#

            # should be updated now
#            left_child = node.children[0]
#            right_child = node.children[1]
#            if left_child.cluster_label == right_child.cluster_label and left_child.cluster_label is not None:
#                node.children = []
#                node.cluster_label = left_child.cluster_label
#                return node
#
#        return node        

In [2]:
MAX_LEAFS = 32

fbe_path = "/home/adrian/workspace/FBE output/maxTopwords_8_tags_maxClasses1024_N2000"

print("Load data file: {}".format(fbe_path))
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sentences = spark.read.load(fbe_path+"/phrases/")
df_short = sentences.select("id", "tokens", "index")
print(df_short.where(col("id") == 22771646).show())
print("N sentences: {}".format(sentences.count()))

print("Load tree..")
nodes = pd.read_json(fbe_path+"/nodes.json", orient="records")
print("nodes: {}".format(nodes.shape))

print("Get list with all possible classes in the sentences file..")
sentences_all_classes, sentences_pd_with_classes = get_list_all_possible_classes(sentences)
print("Number of classes in sentences file: {}".format(len(sentences_all_classes)))
print("Merged dataset with true classes: {}".format(sentences_pd_with_classes.shape))

print("initialise tree..")
treeFBE = Tree(nodes
            , mode="FBE"
            , sentences_all_classes=sentences_all_classes
            , true_classes_all=sentences_pd_with_classes["mesh_ui_diab"])

root = Node(Id=1, depth=0, parent=None, children=[]) # Id = 1 because start at Explorer 
treeFBE.set_build_tree(root)

print("Associate cluster to each sentence..")
sentences_pd_with_classes_uniqueCluster = associate_unique_cluster_to_documents(sentences, treeFBE)
print("Unique clusters in sentences: {}".format(sentences_pd_with_classes_uniqueCluster["uniqueCluster"].nunique())) #####

print("Fit..")
treeFBE.fitTree(treeFBE.tree, sentences_pd_with_classes_uniqueCluster)

pprint(treeFBE.get_performances())

Load data file: /home/adrian/workspace/FBE output/maxTopwords_8_tags_maxClasses1024_N2000


NameError: name 'pyspark' is not defined

In [28]:
treeFBE.tree.children[0].children[0].children[0]

Node id: 4 (depth: 3, cluster_label: 8, children: [5, 6])

In [131]:
#print(609 in sentences_all_classes)
#nodes[["strLinks", "filterValue", "children"]].iloc[365].head()

treeFBE.get_leaf_nodes()

[Node id: 21 (depth: 8, cluster_label: 890, children: []),
 Node id: 48 (depth: 8, cluster_label: 891, children: []),
 Node id: 49 (depth: 5, cluster_label: 105, children: []),
 Node id: 56 (depth: 6, cluster_label: 128, children: []),
 Node id: 71 (depth: 6, cluster_label: 129, children: []),
 Node id: 89 (depth: 5, cluster_label: 16, children: []),
 Node id: 106 (depth: 5, cluster_label: 17, children: []),
 Node id: 173 (depth: 9, cluster_label: 322, children: []),
 Node id: 174 (depth: 9, cluster_label: 323, children: []),
 Node id: 176 (depth: 8, cluster_label: 298, children: []),
 Node id: 177 (depth: 8, cluster_label: 299, children: []),
 Node id: 180 (depth: 6, cluster_label: 45, children: []),
 Node id: 190 (depth: 8, cluster_label: 112, children: []),
 Node id: 191 (depth: 8, cluster_label: 113, children: []),
 Node id: 192 (depth: 7, cluster_label: 73, children: []),
 Node id: 201 (depth: 8, cluster_label: 324, children: []),
 Node id: 202 (depth: 8, cluster_label: 325, child

In [101]:
print(873 in sentences_all_classes)
nodes[["strLinks", "filterValue", "children"]].iloc[375].head()

False


strLinks       {'1': [830, 831]}
filterValue                [791]
children                      []
Name: 375, dtype: object

In [27]:
4
{'F1_macro': 0.8467197871278361,
 'F1_micro': 0.6354458480220039,
 'prec_macro': 0.7356943279744165,
 'prec_micro': 0.4671033336592042,
 'recall_macro': 0.9972114441197784,
 'recall_micro': 0.9935}

6
{'F1_macro': 0.7356349027609322,
 'F1_micro': 0.6202309798501623,
 'prec_macro': 0.5884578696287001,
 'prec_micro': 0.45188113413304254,
 'recall_macro': 0.980985603499415,
 'recall_micro': 0.9885}

8
{'F1_macro': 0.6633409158744573,
 'F1_micro': 0.5998594041757052,
 'prec_macro': 0.5265395307091034,
 'prec_micro': 0.4564950980392157,
 'recall_macro': 0.8961791873294477,
 'recall_micro': 0.8745}

Performance scikit learn: 
{'F1_macro': 0.7182067311303414,
 'F1_micro': 0.2036082121751319,
 'prec_macro': 0.5603293541866284,
 'prec_micro': 0.11334402524839769,
 'recall_macro': 0.9999511409627143,
 'recall_micro': 0.9999105721593247}

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
8
5
6
4
5
6
7
8
6
3
4
5
6
7
8
5
6
7
8
4
5
6
7
8
9
9
7
8
8
9
6
7
5
6
7
8
8
7
2
3
4
5
6
7
8
8
9
10
5
4
5
6
7
7
8
9
10
11
11
10
11
12
11
12
9
10
8
9
10
11
10
11
12
9
6
5
6
7
7
8
6
7
8
8
9
3
4
5
6
6
5
6
6
4
5
6
7
8
7
6
7
8
9
10
11
12
13
8
9
10
11
12
13
14
15
16
17
18
5
6
7
8
8
7
1


18

In [17]:
Nnodes = 0

def get_nodes(node):
    """ get max depth of tree"""
    global Nnodes
    Nnodes += 1
    if len(node.children) == 1:
        get_nodes(node.children[0])
    elif len(node.children) == 2:
        get_nodes(node.children[0])
        get_nodes(node.children[1])

    
get_nodes(treeFBE.tree)
Nnodes

140

In [46]:
nodes.iloc[1]

name                                                            Explorer
tagId                                                                  1
color                                                                NaN
annotations            [{'tokens': ['diabetic'], 'tag': 2, 'from': No...
algo                                             {'value': 'clustering'}
strLinks                                                   {'1': [2, 3]}
strClassPath                                  {'2': [0, 1], '3': [0, 1]}
names                                                                 {}
filterMode                                            {'value': 'anyIn'}
filterValue                                                          [1]
maxTopWords                                                            6
windowSize                                                           NaN
classCenters                                            {'2': 0, '3': 1}
cError                            [0.29197365929895

In [35]:
from collections import Counter
ll = [5,5,4,7,8,4,7,7,7,7,0,2,5, 5, 5]
print(Counter(ll))
print(Counter(ll).most_common()[0][0])
print(Counter(ll).most_common())


Counter({5: 5, 7: 5, 4: 2, 8: 1, 0: 1, 2: 1})
5
[(5, 5), (7, 5), (4, 2), (8, 1), (0, 1), (2, 1)]


In [42]:
class_counts = Counter(ll).most_common()
[c for c, occ in class_counts  if occ == class_counts[0][1]]

[5, 7]